# 5장 오차역전파법

## 5.4 단순한 계층 구현하기

### ex) 사과 쇼핑

- MulLayer : 곱셈 레이어
- AddLayer : 덧셈 레이어

### 5.4.1 곱셈 계층

- 모든 계층은 forward()와 backward() 라는 공통의 메서드를 가짐.
- forward() : 순전파 처리
- backward() :  역전파 처리


In [2]:
class MulLayer:
    def __init__(self): # 인스턴스 변수인 x,y 초기화. 순전파 시의 입력값 유지를 위해 사용.
        self.x = None
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y                
        out = x * y # 두 입력값을 그대로 곱해서 출력

        return out

    def backward(self, dout): # dout는 상류에서 넘어온 미분임.
        dx = dout * self.y  # 순전파 때의 값인 x와 y를 서로 바꿔서 곱함.
        dy = dout * self.x

        return dx, dy

- MulLayer를 이용한 사과쇼핑 구현

In [3]:
apple = 100 # 사과 1개의 가격
apple_num = 2 # 사과의 개수
tax = 1.1 # 소비세

mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward (순전파)
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

# backward (역전파) : 각 변수의 미분을 구할 수 있음.
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("price:", int(price)) #
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)

price: 220
dApple: 2.2
dApple_num: 110
dTax: 200


- backword()의 호출 순서는 forward()때와 반대.
### backword()가 받는 인수는 '순전파의 출력에 대한 미분' 임에 주의할 것!!

### 5.4.2 덧셈 계층

In [4]:
class AddLayer:
    def __init__(self):
        pass # 덧셈 계층에서는 초기화가 필요하지 않기 때문에 아무것도 하지 않음.

    def forward(self, x, y):
        out = x + y # 입력받은 인수를 그냥 더하기만 해서 반환.

        return out

    def backward(self, dout):
        dx = dout * 1 # 상류에서 전달받은 미분을 그대로 하류로 전달.
        dy = dout * 1 

        return dx, dy

### 덧셈과 곱셈의 계층을 사용한 사과 쇼핑 구현

In [5]:
apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# 필요한 계층(layer)들 생성
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# 순전파(forward) 실시
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# 순전파와 반대 순서로 역전파(backward) 실시
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650


-> 이처럼 계산 그래프에서의 계층은 쉽게 구현할 수 있으며, 이를 사용해 복잡한 미분도 계산할 수 있다!

## 5.5 활성화 함수 계층 구현하기

- 신경망에서 사용하는 계층 구현. 신경망을 구성하는 층(계층) 각각을 클래스 하나로 구현.


### 5.5.1 ReLU 계층

- 활성화 함수로 사용되는 ReLU의 수식은 다음과 같음.
---------
y = X (x>0)  
    0 (x<=0)
---------
여기에서 x에 대한 y의 미분은 다음과 같이 구함.

------------
dy / dx = 1 (x>0)  
        0 (x<=0)
--------
--> 순전파 때의 입력인 x가 0보다 크면 역전파는 상류의 값을 하류로 그대로 흘려보내고,
0 이하면 역전파 때 하류로 신호를 보내지 않음.


#### - ReLU 계층 구현 코드

In [7]:
class Relu:
    def __init__(self):
        self.mask = None 
        # mask는 True/False로 구성된 넘파이 배열로,
        # 순전파의 입력인 x의 원소 값이 0 이하면 True, 이상이면 False로 유지.

    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dout[self.mask] = 0 # mask가 True인 곳에서는 상류에서 전파된 dout를 0으로 설정.
        dx = dout

        return dx


### 5.5.2 Sigmoid 계층

- 시그모이드 함수

y = 1 / {1+exp(-x)}

- 'exp' 노드는 y = exp(x) 계산을 수행하고 '/' 노드는 y = 1/x 계산을 수행함.


---

#### 1단계


- '/' 노드 미분 시

dy/dx = -1/x^2 = -y^2



-> 역전파 때는 상류에서 흘러온 값에 -y^2 을 곱해서 하류로 전달.

---

#### 2단계

- '+' 노드는 상류의 값을 여과 없이 하류로 내보냄.

---
#### 3단계

- 'exp' 노드는 y = exp(x) 연산을 수행하며, 그 미분은 다음과 같다.

dy / dx = exp(x)

-> 상류의 값에 순전파 때의 출력을 곱해 하류로 전파함.

---

#### 4단계

- 'x' 노드는 순전파 때의 값을 '서로 바꿔' 곱함.

---

- Sigmoid의 경우, 역전파의 최종 출력이 순전파의 입력인 x와 y만으로 계산이 가능하다.  
따라서 계산의 중간 과정들을 모두 묶어 단순한 'sigmoid' 노드 하나로 대체 할 수 있음.
### -> 간소화하게 되면 중간 계산들을 생략할 수 있어 더 효율적인 계산이고, 노드를 그룹화하여 입력과 출력에만 집중할 수 있다는 점이 중요 포인트임.


- 역전파의 최종 출력물 또한 한번 더 정리하면 y만으로 계산이 가능해짐.  
-> 이처럼 Sigmoid 계층의 역전파는 순전파의 출력(y)만으로 계산이 가능하다.

#### - Sigmoid 계층 코드 구현

In [8]:
class Sigmoid:
    def __init__(self):
        self.out = None

    def forward(self, x):
        out = sigmoid(x) 
        self.out = out
        return out

    # 순전파의 출력을 인스턴스 변수  out에 보관했다가, 역전파때 그 값을 사용함.

    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out

        return dx
